# ***Distribución ESTACIONAMIENTOS***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "puebla" in nombre

    if nombre.startswith("df_") and "puebla" in nombre    
    and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_q_puebla',
 'df_feb_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [4]:
df_may_2024_puebla.columns

Index(['id', 'Q', 'categoria', 'ids', 'Propiedad', 'Precio', 'Metros_total',
       'm2_construido', 'Precio_M2_Terreno', 'Precio_M2_Construido',
       'Personas_interesadas', 'Tiempo_publicacion',
       'Promedio_Interes_Mensual', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Cantidad_de_pisos',
       'Antiguedad', 'Seguridad', 'Fraccionamiento', 'Colonia', 'CP', 'url'],
      dtype='object')

In [5]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido','Estacionamientos':'estacionamiento'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [6]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','estacionamiento']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [7]:
def procesar_dataframes(dataframes_list, nombres_df_filtrados):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)

        # Clasificar 'baño_total' en rangos específicos
        bins = [0, 1, 2, 3, 4, 5, 6, float('inf')]
        labels = ['0', '1', '2', '3', '4', '5', '6']
        df['estacionamiento'] = pd.cut(df['estacionamiento'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'estacionamiento']).size()

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {nombres_df_filtrados[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, nombres_df_filtrados)

Nombre del DataFrame: df_alfa_q_puebla
segmento  estacionamiento
A         0                   0
          1                   9
          2                  33
          3                   1
          4                   0
          5                   0
          6                   0
B         0                   1
          1                  10
          2                  28
          3                   0
          4                   0
          5                   0
          6                   0
C         0                   3
          1                   9
          2                  17
          3                   0
          4                   0
          5                   0
          6                   0
D         0                   0
          1                   5
          2                   2
          3                   0
          4                   0
          5                   0
          6                   0
S         0                   2
       

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2552\2733603970.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2552\2733603970.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_2552\2733603970.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [8]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'estacionamiento'],observed=False).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
estacionamiento     0       1       2      3      4     5      6
segmento                                                        
A                48.0   777.0  1384.0  104.0   44.0  20.0   19.0
B                36.0   879.0  1512.0  100.0   53.0   6.0    8.0
C                48.0  1032.0  1327.0   40.0   11.0   1.0   12.0
D                27.0  1167.0  1166.0   26.0    6.0   1.0    0.0
E                20.0  1324.0  1608.0   21.0    7.0   0.0   29.0
L                 9.0   241.0   284.0   27.0   25.0   6.0   58.0
S                65.0  1311.0  1812.0  273.0  192.0  81.0  181.0


In [9]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['estacionamiento', 'segmento'],observed=False).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_estacionamientos', carpeta='assets/graficas')
    
    return fig

def graficar_tabla(tabla):
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        #title="M2 de Terreno",
        #xaxis_title="Rango de M2",
         yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        legend_title="Estacionamientos",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    #pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
segmento            A     B     C     D       E      L     S
estacionamiento                                             
0                  48    36    48    27    20.0    9.0    65
1                 777   879  1032  1167  1324.0  241.0  1311
2                1384  1512  1327  1166  1608.0  284.0  1812
3                 104   100    40    26    21.0   27.0   273
4                  44    53    11     6     7.0   25.0   192
5                  20     6     1     1     0.0    6.0    81
6                  19     8    12     0    29.0   58.0   181


In [10]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2', '#F79CB9', '#E50CB6', '#4225CF']

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['estacionamiento', 'segmento'],observed=False).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_pie_estacionamientos', carpeta='assets/graficas')
    
    return fig

def graficar_tabla(tabla):
    # Sumar los valores para cada estacionamiento
    suma_estacionamientos = tabla.sum(axis=1)
    
    fig = go.Figure(data=[go.Pie(
        labels=suma_estacionamientos.index,
        values=suma_estacionamientos.values,
        marker=dict(colors=colores[:len(suma_estacionamientos)]),
        #hole=.3  # Si deseas un gráfico de pastel tipo "donut"
    )])
    
    fig.update_layout(
        #title="Distribución de Estacionamientos por Segmento",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    )
    
    fig.show()
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

fig = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
segmento            A     B     C     D       E      L     S
estacionamiento                                             
0                  48    36    48    27    20.0    9.0    65
1                 777   879  1032  1167  1324.0  241.0  1311
2                1384  1512  1327  1166  1608.0  284.0  1812
3                 104   100    40    26    21.0   27.0   273
4                  44    53    11     6     7.0   25.0   192
5                  20     6     1     1     0.0    6.0    81
6                  19     8    12     0    29.0   58.0   181
